In [ ]:
!pip install openpyxl

In [ ]:
import pandas as pd
import xlwings as xw 
import openpyxl
from typing import List
from datetime import datetime, date

In [653]:

class ETL_system:
    def __init__(self):
        pass
    def _read_excel_file(self, filename:str)-> pd.ExcelFile:  
        #deal with exception like FileNotFoundError
        return  pd.ExcelFile(filename)
    
    def get_sheets_name(self, filename:str)-> List[str]:
        file=self._read_excel_file
        f=self._read_excel_file(filename)
        return f.sheet_names
    
    def read_one_sheet(self, filename:str, sheet_names:str) -> pd.DataFrame:
        f=self._read_excel_file(filename)
        df=pd.read_excel(f, sheet_names)  
        #delete all empty columns (perminantly) from the excel-file if found
        self.remove_empty_cols(df)
        return  df 
    
    def get_column(self, filename:str, sheet_names:str,col_name:str):
        return self.read_one_sheet(filename, sheet_names)[col_name]

    #detect empty columns in a df and remove them if found
    def remove_empty_cols(self, df:pd.DataFrame)->pd.DataFrame:
        empty_cols=list(df.filter(regex='Unnamed:'))
        df.drop(empty_cols, axis=1,inplace=True)
        return df
    
    def check_douplicated(self, fname:str, sheet_name:str, column_names:list)->bool:
        #parm. column_names is a list of  one (values of a column) or more column names.
        #If one column the function check douplicated values of that spesific colum
        df=self.read_one_sheet(fname,sheet_name)
        return 0<df.duplicated(subset=column_names).sum()
    
    def get_douplicated(self, fname:str, sheet_name:str, column_names:list):#->bool:
        #colum is a list of  one (values of a column) or more column names.
        #If one column the function check douplicated values of that spesific colum
        df=self.read_one_sheet(fname,sheet_name)
        return df[df.duplicated(subset=column_names)]#[column_names]
    
    #update the duplicated values of kolonne Medlemmer Medlemsnummer with unique vals (perminantly of the excel-file)
    def fix_dublicated(self)->None:
        #duplicated Medlemsnummer in sheet "Medlemmer"
        duplicated_ids_Medlemmer=self.get_douplicated("Datagrunnlag.xlsx", "Medlemmer",["Medlemsnummer"])
        #get their indices to use them when dealing with 
        indices_of_duplicated=duplicated_ids.index
        
        mnrs_column=self.get_column("Datagrunnlag.xlsx", "Medlemmer",["Medlemsnummer"])
        #get all values of kolonne  Medlemnummer
        all_mnrs=list(mrs_column.values)

        #temp var.
        mx = int(max(all_mnrs))
        f = openpyxl.load_workbook("Datagrunnlag.xlsx") #open for writing
        sheet = f["Medlemmer"]

        for i in list(indices_of_duplicated):  
            
            ny_id=mx + 1 #generate an uniq_id of kolonne "Medlemmernummer"
            sheet.cell(row=i+2, column=1).value=ny_id #update the duplicated value with the unique key (perminantly in the excel-file)
            mx=ny_id #save the max value 
        #save changes by overrriding the xlsx-file
        f.save("Datagrunnlag.xlsx")
        
    def change_sheet_name(self, file, old_sh_name, new_sh_name)->None:
        f = openpyxl.load_workbook(file) #open for writing
        sh = f[old_sh_name]
        sh.title = new_sh_name
        f.save(file)
    
    #fill missing vals in a column with the most frequent value
    def fill_missing_vals(self,file,sheet,column)->None:
        df=self.read_one_sheet(file, sheet)
        mest_freq_val=df[column].mode()[0]
        f = openpyxl.load_workbook(file) #open for writing 
        sh = f[sheet]
        i=etl.read_one_sheet("Datagrunnlag.xlsx", "Medlemmer").tail(1).index[0]
        sh.cell(row=i+2, column=9).value=mest_freq_val
        
        f.save(file)
    #calculate one age given a birth date
    def _calculate_age(self,d:str)->int:
        year=int(d[-4:])
        now=date.today().year
        age=now-year
        return age
    #calculate ages given a series of birth dates
    def get_ages(self, dates:pd.Series)->pd.Series:
        return dates.apply(self._calculate_age)
    
    def to_capitized(self, col:pd.Series):   
        return col.str.capitalize()
    
    def to_lower(self, col:pd.Series):
        return col.str.lower()
 
etl=ETL_system()


In [445]:
duplicated_df=etl.get_douplicated("Datagrunnlag.xlsx", "Medlemmer",["Medlemsnummer"])

duplicated_df

,Medlemsnummer,Fornavn,Etternavn,Fødselsdato,Kjønn,Medlemstype,Gateadresse,Postnummer,Poststed
59,42,Ludvig,LARSEN,08.02.1908,M,veteran,Frendstadgjelen,6065,Ulsteinvik
80,42,Hansd,RASMUSSEN,19.02.1947,M,senior,Støa,6639,Torvikbukt
86,42,Karl,SOLBERG,10.09.1941,M,veteran,Storinnsjøveien,6390,Vestnes


In [448]:
indices_of_duplicated=duplicated_df.index
etl.fix_dublicated()
df=etl.read_one_sheet("Datagrunnlag.xlsx", "Medlemmer")
df.iloc[indices_of_duplicated]


,Medlemsnummer,Fornavn,Etternavn,Fødselsdato,Kjønn,Medlemstype,Gateadresse,Postnummer,Poststed
59,100,Ludvig,LARSEN,08.02.1908,M,veteran,Frendstadgjelen,6065,Ulsteinvik
80,101,Hansd,RASMUSSEN,19.02.1947,M,senior,Støa,6639,Torvikbukt
86,102,Karl,SOLBERG,10.09.1941,M,veteran,Storinnsjøveien,6390,Vestnes


In [458]:

etl.change_sheet_name("Datagrunnlag.xlsx", "kontingent ", "Kontigent")

#obs. Du vil få error fordi denne metoden ble kalt første  og kolonne "kontingent " har fått en nytt navn!

KeyError: 'Worksheet kontingent  does not exist.'

In [470]:

#see last val.of column "Poststed" which has a missing val.
etl.read_one_sheet("Datagrunnlag.xlsx", "Medlemmer").tail(1)


,Medlemsnummer,Fornavn,Etternavn,Fødselsdato,Kjønn,Medlemstype,Gateadresse,Postnummer,Poststed
98,90,Marie,NaN,12.08.1964,F,senior,Myrvangveien,6601,NaN


In [497]:
#fille manglende info
etl.fill_missing_vals("Datagrunnlag.xlsx","Medlemmer","Poststed")
etl.read_one_sheet("Datagrunnlag.xlsx", "Medlemmer").tail(1)

,Medlemsnummer,Fornavn,Etternavn,Fødselsdato,Kjønn,Medlemstype,Gateadresse,Postnummer,Poststed
98,90,Marie,NaN,12.08.1964,F,senior,Myrvangveien,6601,Sylte


In [465]:
df3=etl.read_one_sheet("Datagrunnlag.xlsx", "Medlemmer")



In [78]:
#sjekke om fult navn(fornavn + etternavn) er duplicated
etl.check_douplicated("Datagrunnlag.xlsx", "Medlemmer",["Fornavn","Etternavn"])

False

In [661]:
#fix capital letters for "Etternavn"
meldemmer=etl.read_one_sheet("Datagrunnlag.xlsx", "Medlemmer")
lowered=etl.to_lower(meldemmer["Etternavn"])
capitized=etl.to_capitized(lowered)
capitized

0            Aas
1          Aasen
2     Abrahamsen
3          Ahmed
4            Ali
         ...    
94     Thomassen
95       Thorsen
96         Tveit
97           Vik
98           NaN
Name: Etternavn, Length: 99, dtype: object

In [662]:
#calculate ages
birth_dates=etl.read_one_sheet("Datagrunnlag.xlsx", "Medlemmer")["Fødselsdato"]

alder=etl.get_ages(birth_dates)
alder.rename("alder", inplace=True)

medlemmer_df=etl.read_one_sheet("Datagrunnlag.xlsx", "Medlemmer")
medlemmer_df["Medlemstype"]=etl.to_capitized(medlemmer_df["Medlemstype"])
merged_medlemmer_df=pd.merge(medlemmer_df,alder,left_index=True,right_index=True)

merged_medlemmer_df

,Medlemsnummer,Fornavn,Etternavn,Fødselsdato,Kjønn,Medlemstype,Gateadresse,Postnummer,Poststed,alder
0,62,Sahra,AAS,28.03.1957,F,Veteran,Nytrøa,6166,Sæbø,65
1,87,Hanne,AASEN,22.05.1964,F,Senior,Krokveien,6399,Vestnes,58
2,99,Solveig,ABRAHAMSEN,08.05.1968,F,Senior,Langsæterlia,6687,Valsøyfjorden,54
3,71,Karen,AHMED,23.07.1960,F,Veteran,Rødalsveien,6239,Sykkylven,62
4,52,Emma,ALI,05.11.1951,F,Veteran,Sottdalen,6051,Valderøy,71
...,...,...,...,...,...,...,...,...,...,...
94,66,Maya,THOMASSEN,11.01.1958,F,Senior,Sørsjølia,6210,Sylte,64
95,81,Susanne,THORSEN,24.05.2005,F,Junior,Klettvangveien,6310,Veblungsnes,17
96,97,Anne,TVEIT,30.01.1968,F,Senior,Skibakkveien,6683,Vågland,54
97,78,Åse,VIK,25.06.1962,F,Senior,Svartberget,6280,Søvik,60


In [663]:


alg=etl.read_one_sheet("Datagrunnlag.xlsx", "Kontigent")["Aldersgruppe"]
splited_ranges=alg.str.split(r"-| ", n=-1,expand=True)
splited_ranges.columns = ['min', 'mx', '']

Kontigent_df=etl.read_one_sheet("Datagrunnlag.xlsx", "Kontigent")

splited_ranges["mx"]=splited_ranges["mx"].replace(to_replace = '+', value = "120")

merged_kontigent_df=pd.merge(Kontigent_df,splited_ranges,left_index=True,right_index=True)
merged_kontigent_df

,Medlemstype,kontingent,Periode,Aldersgruppe,min,mx,
0,Senior,900,2017,18-60,18,60,None
1,Junior,400,2017,10-17,10,17,None
2,Veteran,750,2017,60 +,60,120,
3,Senior,850,2016,18-60,18,60,None
4,Junior,350,2016,10-17,10,17,None
5,Veteran,700,2016,60 +,60,120,


In [664]:
#steg 3: premisser/joine

#join based on medlemstype
pd.merge(merged_kontigent_df,merged_medlemmer_df,on=["Medlemstype"])

#limit to "alder" between "min" and "max"

#merged_medlemmer_df["alder"].between(merged_kontigent_df["min"].astype(int), merged_kontigent_df["mx"].astype(int))

#pd.merge(merged_kontigent_df,merged_medlemmer_df,on=[merged_medlemmer_df["alder"].between(merged_kontigent_df["min"], merged_kontigent_df["mx"])])


,Medlemstype,kontingent,Periode,Aldersgruppe,min,mx,,Medlemsnummer,Fornavn,Etternavn,Fødselsdato,Kjønn,Gateadresse,Postnummer,Poststed,alder
0,Senior,900,2017,18-60,18,60,None,87,Hanne,AASEN,22.05.1964,F,Krokveien,6399,Vestnes,58
1,Senior,900,2017,18-60,18,60,None,99,Solveig,ABRAHAMSEN,08.05.1968,F,Langsæterlia,6687,Valsøyfjorden,54
2,Senior,900,2017,18-60,18,60,None,76,Rigmor,ANTONSEN,11.12.1961,F,Gjeråsen,6264,Tennfjord,61
3,Senior,900,2017,18-60,18,60,None,74,Ingeborg,ARNESEN,17.11.1960,F,Ryan,6260,Skodje,62
4,Senior,900,2017,18-60,18,60,None,96,Guri,AUNE,22.05.1966,F,Moanlangsætra,6656,Surnadal,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,Veteran,700,2016,60 +,60,120,,46,Daniel,SOLHEIM,26.05.1949,M,Langtjønna,6656,Surnadal,73
184,Veteran,700,2016,60 +,60,120,,30,Olaf,STRAND,10.09.1940,M,Grøntjønnveien,6283,Vatne,82
185,Veteran,700,2016,60 +,60,120,,31,Nils,SVENDSEN,12.09.1940,M,Russuveien,6310,Veblungsnes,82
186,Veteran,700,2016,60 +,60,120,,61,Norah,SÆTHER,06.04.1956,F,Dallvangen,6165,Sæbø,66


In [615]:
betalinger_df=etl.read_one_sheet("Datagrunnlag.xlsx", "Betalinger")

betalte_meldemmer=pd.merge(medlemmer_df,betalinger_df,on=["Medlemsnummer"])

print(f"Ant. betalte medlemmer er:{len(betalte_meldemmer)} men det er {len(medlemmer_df)-len(betalte_meldemmer)} ikke betalte.")


Ant. betalte medlemmer er:86 men det er 13 ikke betalte.


In [568]:


"""
doing:

- Lag kode (i valgfritt språk) som setter korrekt medlemstype utfra krav til alder,
og et som sjekker om rett innbetaling er utført.


done:
-sjekk hvis to navn er doublettet "fornavn + etternavn"
- håntere dubletter (Medlemsnummer i to fliker)
-fylle manglende med gjennomsnittlige verdi av kolonner
-ekstrahere datoer (dag, mnd, år)
-ranges/ alder av 
-missing Id 21 30 31 32 46 61 66 67 78 81 86 89 90 97
- edit page name which looks like "kontingent "



to do:

- medlemmer som er registrert med feil medlemstype
- medlemmer som har betalt gal kontingent.
- medlemmer som er registrert med feil medlemstype og har betalt gal kontingent
- rapport over medlemmer og innbetalinger


observasjoner/fakum:
-double id ()
-små bokstav for sheet-nv"kontigent"

ideer/deloppgaver:
-Etter navn til små bokstaver så capital letters
-køjnn til små bokstaver

sjekk double navn eller doubel medlemnr.
-sjekk folk som er over 110 år (eldste menneske i norger Marie Antonette Andersen som var 110) 
-medlemer som betate to ganger (3 medlemer deler samme nøkkel)
-thre medlemmer som hadde
-statistics som gjennomsnitt verdi/ mn /variance 

result/anayses:
-ingen flik   har duplicated rader (columner)
-ingen fullt navn er duplicated i Medlemmer


sprs:
Hva burde tenkes på under planleggingsarbeidet
riktig verktøyer-pandas (enkel å bruke raskt (c språk))
--laget en klasse ..... n funksjoner for E ----n funksjoner for t ... n funksjoner for l

• Valg av datamodell
• Hva som kan være viktig å passe på ved innlesing fra Excel-arkene
-datatyper
-om det er mer enn enn sheet
-tomme sider
- verdier kolonner har datatyper
• Hvordan rader med feil bør håndteres
-spørs kolnner type for eks. Medlemmernr.:
måter: 1) fjerne  2)fylles med gjennomsnittlige verdi av colums 3) i vårt tilfelle
• Hvordan eventuelle dubletter eller manglende informasjon bør håndteres
fjernes, gjennomsnittlige verdier(alder), mest freqvente klasse (Poststed-kolonne-Medlemmer)
• Hvor i prosessen og hvordan det er fornuftig å vaske data, hvis det skal gjøres
etternavn
-scenario 1: spørs
-mens/rett at man leser en sheet kan det være lurt å fjerne tomme kolonner --- 
-scenario 2: fra begynelsen av database-oprettelse (sette strenge regler da unngå man feil)
 (primarynøkkel(Medlemsnummer) er unik, entiteter NOT NULL,)
• Analyser datasettene

"""




'\n\ndone:\n- håntere dubletter (Medlemsnummer i to fliker)\n\ndoing:\n\n- Lag kode (i valgfritt språk) som setter korrekt medlemstype utfra krav til alder,\nog et som sjekker om rett innbetaling er utført.\n\nto do:\n\n- medlemmer som er registrert med feil medlemstype\n- medlemmer som har betalt gal kontingent.\n- medlemmer som er registrert med feil medlemstype og har betalt gal kontingent\n- rapport over medlemmer og innbetalinger\n\n\nobservasjoner/fakum:\n-double id ()\n-små bokstav for sheet-nv"kontigent"\n\nideer/deloppgaver:\n-Etter navn til små bokstaver så capital letters\n-køjnn til små bokstaver\n-sjekk hvis to navn er doublettet "fornavn + etternavn"\nsjekk double navn eller doubel medlemnr.\n-sjekk folk som er over 110 år (eldste menneske i norger Marie Antonette Andersen som var 110) \n-medlemer som betate to ganger (3 medlemer deler samme nøkkel)\n-thre medlemmer som hadde\n-statistics like gjennomsnitt verdi/ mn /variance \n-ranges/ alder av \n-fylle manglende med gje

In [ ]:
#ikke ferdig skrevet kode


"""def calculate_age(born):
    born = datetime.strptime(born, "%d.%m.%Y").date()
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month,   born.day))"""

"""def calculate_ages(dates_col:pd.DataFrame):
    #colum_name=dates_col.columns
    dates_col.apply(calculate_age)
    df['Age']= df[dates_col].apply(calculate_age)
    return df['Age']
#display(df)

dates=etl.read_one_sheet("Datagrunnlag.xlsx", "Medlemmer")["Fødselsdato"]
calculate_ages(dates)"""
"""dates=etl.read_one_sheet("Datagrunnlag.xlsx", "Medlemmer")["Fødselsdato"]
calculate_ages(dates)

def conv_to_period(date:str):
    day, month, year = map(int, date.split("."))
    return pd.Period(year=year, month=month, day=day, freq="D")

def calculate_age(d:pd.Period)->int:
    age=date.today()-d.year
    return int(age)

df_dates=etl.read_one_sheet("Datagrunnlag.xlsx", "Medlemmer")["Fødselsdato"]

#dates=df_dates.apply(calculate_age)
#calculate_ages(dates)
#print(df["Dates"])

#calculate_ages(dates)

"""